### Importando as dependecias

In [1]:
import DecisionTree_entropy
import csv
from pyspark.sql.types import StringType
import random
from math import log, sqrt
import logging

In [2]:
sc.addPyFile("DecisionTree_entropy.py")

In [3]:
def csv_to_rdd(filepath):
    rdd = sc.textFile(filepath).mapPartitions(lambda line: csv.reader(line, delimiter=','))
    rdd_2 = rdd.map(lambda row: (int(row[0]), row[1], row[2], int(row[3]), row[4],row[5], row[6], row[7], row[8], int(row[9]),int(row[10]), int(row[11]), row[12], row[13]))
    rdd_3 = rdd_2.map(lambda x: x.strip() if isinstance(x, str) else x)
    return rdd_3

### Testando a árvore da classe DecisionTreeClassifier (DecisionTree_entropy.py)

In [4]:
def test_tree():
    data = csv_to_rdd("../data/income.csv").collect()
    tree = DecisionTree_entropy.DecisionTreeClassifier(random_features=True)
    tree.fit(data)

    print(tree.predict([39, 'State-gov', 'Bachelors', 13, 'Never-married',
                        'Adm-clerical', 'Not-in-family', 'White', 'Male',
                        2174, 0, 40, 'United-States']))


if __name__ == '__main__':
    test_tree()

 <=50K


### Função que cria lista de listas em cada partição

In [5]:
def dataset_tree(partition):
    final_iterator = []
    for sub_list in partition:
        final_iterator.append(tuple(sub_list))
    return iter(final_iterator)

### Lendo o dataset e quebrando ele em 60 partições (árvores do random forest)

In [6]:
data = csv_to_rdd("../data/income.csv").repartition(60)

### Criando um rdd com 60 partições do tipo [indice da partição, ((linha 1),(linha 2)...)]

In [7]:
rdd_trees = data.mapPartitionsWithIndex(lambda index, part: (yield index, list(dataset_tree(part))))

In [8]:
rdd_trees.first()

(0,
 [(42,
   ' ?',
   ' HS-grad',
   9,
   ' Married-civ-spouse',
   ' ?',
   ' Husband',
   ' White',
   ' Male',
   0,
   0,
   40,
   ' United-States',
   ' <=50K'),
  (55,
   ' ?',
   ' HS-grad',
   9,
   ' Married-civ-spouse',
   ' ?',
   ' Wife',
   ' Asian-Pac-Islander',
   ' Female',
   0,
   0,
   40,
   ' United-States',
   ' <=50K'),
  (60,
   ' Private',
   ' 7th-8th',
   4,
   ' Never-married',
   ' Transport-moving',
   ' Not-in-family',
   ' White',
   ' Male',
   0,
   0,
   48,
   ' United-States',
   ' >50K'),
  (31,
   ' Private',
   ' Bachelors',
   13,
   ' Married-civ-spouse',
   ' Prof-specialty',
   ' Husband',
   ' White',
   ' Male',
   7688,
   0,
   40,
   ' Canada',
   ' >50K'),
  (23,
   ' Private',
   ' Bachelors',
   13,
   ' Never-married',
   ' Adm-clerical',
   ' Own-child',
   ' White',
   ' Female',
   0,
   0,
   40,
   ' United-States',
   ' <=50K'),
  (27,
   ' Private',
   ' HS-grad',
   9,
   ' Never-married',
   ' Sales',
   ' Own-child',
   

### Tentando usar a classe DecisionTree_entropy - Não consigo retornar o "objeto" DecisionNode da classe DecisionTreeClassifier (DecisionTree_entropy.py)

In [28]:
tree = DecisionTree_entropy.DecisionTreeClassifier(max_depth=-1)
rdd_trees.map(lambda x: (x[0],tree.fit(x[1]))).collect()

[(0, None),
 (1, None),
 (2, None),
 (3, None),
 (4, None),
 (5, None),
 (6, None),
 (7, None),
 (8, None),
 (9, None),
 (10, None),
 (11, None),
 (12, None),
 (13, None),
 (14, None),
 (15, None),
 (16, None),
 (17, None),
 (18, None),
 (19, None),
 (20, None),
 (21, None),
 (22, None),
 (23, None),
 (24, None),
 (25, None),
 (26, None),
 (27, None),
 (28, None),
 (29, None),
 (30, None),
 (31, None),
 (32, None),
 (33, None),
 (34, None),
 (35, None),
 (36, None),
 (37, None),
 (38, None),
 (39, None),
 (40, None),
 (41, None),
 (42, None),
 (43, None),
 (44, None),
 (45, None),
 (46, None),
 (47, None),
 (48, None),
 (49, None),
 (50, None),
 (51, None),
 (52, None),
 (53, None),
 (54, None),
 (55, None),
 (56, None),
 (57, None),
 (58, None),
 (59, None)]

### Recriando a arvore para retornar um dicionário no lugar de um DecisionNode da classe DecisionTreeClassifier (DecisionTree_entropy.py)

In [9]:
def choose_random_features(row):
    num_features = len(row) - 1
    return random.sample(range(num_features), int(sqrt(num_features)))

In [10]:
def get_features_subset(row, features_indexes):
    return [row[i] for i in features_indexes]

In [11]:
def unique_counts(rows):
    results = {}
    for row in rows:
        r = row[len(row) - 1]
        if r not in results:
            results[r] = 0
        results[r] += 1
    return results

In [12]:
def entropy(rows):
    log2 = lambda x: log(x) / log(2)
    results = unique_counts(rows)
    ent = 0.0
    for r in results.keys():
        p = float(results[r]) / len(rows)
        ent = ent - p * log2(p)
    return ent

In [13]:
def divide_set(rows, column, value):
    split_function = None
    if isinstance(value, int) or isinstance(value, float):
        split_function = lambda row: row[column] >= value
    else:
        split_function = lambda row: row[column] == value

    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]

    return set1, set2

### returna dicionario

In [14]:
def build_tree(rows,depth):
    if len(rows) == 0:
        return 0
    if depth == 0:
        return 1

    current_score = entropy(rows)
    best_gain = 0.0
    best_criteria = None
    best_sets = None
    column_count = len(rows[0]) - 1
    
    col=-1
    value=None
    results=None
    tb=None
    fb=None

    for col in range(0, column_count):
        column_values = {}
        for row in rows:
            column_values[row[col]] = 1
        for value in column_values.keys():
            set1, set2 = divide_set(rows, col, value)

            p = float(len(set1)) / len(rows)
            gain = current_score - p * entropy(set1) - (1 - p) * entropy(set2)
            if gain > best_gain and len(set1) > 0 and len(set2) > 0:
                best_gain = gain
                best_criteria = (col, value)
                best_sets = (set1, set2)

    if best_gain > 0:
        trueBranch = build_tree(best_sets[0], depth - 1)
        falseBranch = build_tree(best_sets[1], depth - 1)
        return {col:best_criteria[0], value:best_criteria[1], results:None, tb:trueBranch, fb:falseBranch}
    else:
        return {col:-1, value:None, results:unique_counts(rows), tb:None, fb:None}

In [15]:
def teste_fit(rows, features_indexes):
    if len(rows) < 1:
        raise ValueError("Nao ha amostras suficientes no dataset de entrada.")

    #features_indexes = choose_random_features(rows[0])
    rows = [get_features_subset(row, features_indexes) + [row[-1]] for row in rows]

    return build_tree(rows,-1)

In [16]:
def classify(observation, tree):
    if tree['results'] is not None:
        return list(tree['results'].keys())[0]
    else:
        v = observation[tree['col']]
        branch = None
        if isinstance(v, int) or isinstance(v, float):
            if v >= tree['value']:
                branch = tree['tb']
            else:
                branch = tree['fb']
        else:
            if v == tree['value']:
                branch = tree['tb']
            else:
                branch = tree['fb']
        return classify(observation, branch)

In [17]:
def tree_predict(features, features_indexes, tree):
    if not all(i in range(len(features))
            for i in features_indexes):
        raise ValueError("As variaveis passadas nao batem com o conjunto utilizado para treino")
    features = get_features_subset(features, features_indexes)

    return classify(features, tree)

In [41]:
def predict(trees, feature, features_indexes):
    predictions = []

    for tree in trees:
        predictions.append(tree_predict(feature, features_indexes, tree))

    return max(set(predictions), key=predictions.count)

### Agora o map retorna a árvore, porém aparentemente não será possível acessar como um dicionário

In [50]:
features_indexes = choose_random_features(rdd_trees.first()[1][0])
rdd_trees.map(lambda x: teste_fit(x[1], features_indexes)).collect()

[{2: 2,
  ' Other-relative': ' Husband',
  None: {2: 1,
   ' Other-relative': ' Masters',
   None: {2: 2,
    ' Other-relative': ' Wife',
    None: {2: 1,
     ' Other-relative': ' Bachelors',
     None: {2: 1,
      ' Other-relative': ' 7th-8th',
      None: {2: 2,
       ' Other-relative': ' Not-in-family',
       None: {2: 1,
        ' Other-relative': ' Some-college',
        None: {2: -1, ' Other-relative': None, None: None}}}}}}}},
 {2: 2,
  ' Other-relative': ' Husband',
  None: {2: 1,
   ' Other-relative': ' Masters',
   None: {2: 1,
    ' Other-relative': ' Bachelors',
    None: {2: 1,
     ' Other-relative': ' Doctorate',
     None: {2: 2,
      ' Other-relative': ' Wife',
      None: {2: 1,
       ' Other-relative': ' Assoc-voc',
       None: {2: 0,
        ' Other-relative': ' Other',
        None: {2: 1,
         ' Other-relative': ' 11th',
         None: {2: 2,
          ' Other-relative': ' Not-in-family',
          None: {2: -1, ' Other-relative': None, None: None}}}}}}

### a função predic abaixo precisa dos resultados das árvores como parametro

In [51]:
ressult_trees = rdd_trees.map(lambda x: teste_fit(x[1], features_indexes)).collect()

In [52]:
teste = data.sample(False,0.3)

In [53]:
teste.map(lambda x: (x[:-1], x[-1])).take(10)

[((31,
   ' Private',
   ' Bachelors',
   13,
   ' Married-civ-spouse',
   ' Prof-specialty',
   ' Husband',
   ' White',
   ' Male',
   7688,
   0,
   40,
   ' Canada'),
  ' >50K'),
 ((39,
   ' Private',
   ' HS-grad',
   9,
   ' Married-civ-spouse',
   ' Transport-moving',
   ' Husband',
   ' White',
   ' Male',
   0,
   0,
   60,
   ' United-States'),
  ' <=50K'),
 ((49,
   ' Local-gov',
   ' Bachelors',
   13,
   ' Divorced',
   ' Prof-specialty',
   ' Not-in-family',
   ' Black',
   ' Female',
   0,
   0,
   40,
   ' United-States'),
  ' <=50K'),
 ((31,
   ' Private',
   ' Some-college',
   10,
   ' Married-civ-spouse',
   ' Machine-op-inspct',
   ' Husband',
   ' Black',
   ' Male',
   0,
   0,
   40,
   ' Trinadad&Tobago'),
  ' <=50K'),
 ((20,
   ' Private',
   ' HS-grad',
   9,
   ' Never-married',
   ' Handlers-cleaners',
   ' Not-in-family',
   ' White',
   ' Male',
   0,
   0,
   45,
   ' United-States'),
  ' <=50K'),
 ((25,
   ' Private',
   ' 12th',
   8,
   ' Never-marrie

### Ver funções predict e classify mais acima.
#### Em classify eu tenco percorrer cada árvore como se fosse um dicionário, mas dá erro

In [54]:
my_predict = teste.map(lambda x: (predict(ressult_trees,x[:-1], features_indexes), x[-1]))

In [55]:
my_predict.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 44.0 failed 1 times, most recent failure: Lost task 0.0 in stage 44.0 (TID 379, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/fernandaborgesdasilva/Documents/Fernanda/Mestrado/spark-2.3.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/Users/fernandaborgesdasilva/Documents/Fernanda/Mestrado/spark-2.3.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/fernandaborgesdasilva/Documents/Fernanda/Mestrado/spark-2.3.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/fernandaborgesdas